In [1]:
import sys, os
sys.path.append('../src')
import numpy as np
from scipy.optimize import differential_evolution
from numba import jit
import time, multiprocessing, pickle

from NeutrinoFuncs import BinnedNeutrinoRates, BinnedNeutrinoRates2
from WIMPFuncs import BinnedWIMPRate,MeanInverseSpeed_SHM,C_SI, BinnedWIMPRate2
from LabFuncs import FormFactorHelm
from Params import *
#==============================================================================#
ne = 50 # number of energy bins (anything >50 is accurate enough)
nm = 200 # number of mass points
Flux_norm = NuFlux # See Params.py
Flux_err = NuUnc # See Params.py
E_th = 1.0e-4 # Threshold
E_max = 200.0 # Max recoil energy

Nuc = Xe131

@jit(nopython=True)
def H0Func(datObs, RNu, nuiList, Uncs, exposure):
    temp = nuiList@RNu
    return - exposure*np.sum(datObs*(np.log(exposure)+np.log(temp))-temp)+ 0.5*np.sum(((1-nuiList)/Uncs)**2)

@jit(nopython=True)
def H1Func(datObs, RNu, RWIMP, mu, nuiList, Uncs, exposure):
    temp = nuiList@RNu + RWIMP*mu
    return - exposure*np.sum(datObs*(np.log(exposure)+np.log(temp))-temp) + 0.5*np.sum(((1-nuiList)/Uncs)**2)


class MCCheck(object):
    
    def __init__(self, mDM, E_th, E_max, ne):
        nunames, self._NuUnc, self._R_nu = BinnedNeutrinoRates2(E_th,E_max,ne,Nuc,Flux_norm)
        self._RWIMP = BinnedWIMPRate2(E_th,E_max,ne,np.array([mDM]),Nuc,C_SI,\
                                      FormFactorHelm,MeanInverseSpeed_SHM)[1][0]
        self._selList = np.linspace(0,len(nunames)-1,len(nunames),dtype=int)
        return None
    
    def _datGenH0(self, RNu, Uncs, exposure):
        for i in range(100):
            nuiList = np.array(list(map(lambda d: np.random.normal(loc=1.,scale=d), Uncs)))
            if np.all(nuiList>0):
                break
        dat = nuiList@RNu
        return np.array(list(map(lambda d: np.random.poisson(lam=d), dat*exposure)))/exposure
    
    def _datGenH1(self, RNu, RWIMP, Uncs, exposure):
        for i in range(100):
            nuiList = np.array(list(map(lambda d: np.random.normal(loc=1.,scale=d), Uncs)))
            if np.all(nuiList>0):
                break
        dat = nuiList@RNu + RWIMP
        return np.array(list(map(lambda d: np.random.poisson(lam=d), dat*exposure)))/exposure
    
    def _statisticGen(self, datObs, RNu, RWIMP, Uncs, exposure):
        n = np.shape(Uncs)[0]
        H0Gen = lambda x: H0Func(datObs, RNu, x, Uncs, exposure)
        H1Gen = lambda x: H1Func(datObs, RNu, RWIMP, x[0], x[1:], Uncs, exposure)

        boundsH0 = np.transpose([np.zeros(n)+1.0e-5, np.ones(n)*3.])
        boundsH1 = np.transpose([np.zeros(n+1)+1.0e-5, np.append(array([1.0e2]),np.ones(n)*3.)])
        likeDeno = differential_evolution(H1Gen, boundsH1, tol=0, atol=0.01)
        likeNume = differential_evolution(H0Gen, boundsH0, tol=0, atol=0.01)
        #print(likeDeno," ",likeNume," ",[datDM])
        return [2*(likeNume.fun-likeDeno.fun), likeNume.x, likeDeno.x, datObs] 
    
    def H0StatGen(self, sigma0, exposure, n, selList=[]):
        if selList == []:
            selList = self._selList
        RNu = self._R_nu[selList]
        NuUncs = self._NuUnc[selList]
        iEnd = np.max([len(x[x>1.0e-10]) for x in RNu])
        RNu = RNu[:,:iEnd]
        RWIMP = self._RWIMP[:iEnd]*10**(sigma0+45.)
        #datDM = self._sigGen(1.)[:iEnd]*10**(sigma0+45.)
        
        datObsList = np.array(list(map(lambda i: \
                self._datGenH0(RNu, RWIMP, NuUncs, exposure), range(n))))
        time_start = time.perf_counter()
        pool = multiprocessing.Pool(4)
        multiple_results = [pool.apply_async(\
                self._statisticGen, (datObsList[i], RNu, sigma0, NuUncs, exposure))\
                                             for i in range(n)]
        pool.close()
        pool.join()
        H0Stat= [res.get() for res in multiple_results]
        #H1Stat = [self._statisticGen(datObsList[i], RNu, sigma0, Uncs, exposure) for i in range(n)]
        time_end = time.perf_counter()
        print("Time costed: {0} s.".format(time_end-time_start))
        return H0Stat
    
    def H1StatGen(self, sigma0, exposure, n, selList=[]):
        if selList == []:
            selList = self._selList
        RNu = self._R_nu[selList]
        NuUncs = self._NuUnc[selList]
        iEnd = np.max([len(x[x>1.0e-10]) for x in RNu])
        RNu = RNu[:,:iEnd]
        RWIMP = self._RWIMP[:iEnd]*10**(sigma0+45.)
        
        datObsList = np.array(list(map(lambda i: \
                self._datGenH1(RNu, RWIMP, NuUncs, exposure), range(n))))
        time_start = time.perf_counter()
        pool = multiprocessing.Pool(4)
        multiple_results = [pool.apply_async(\
                self._statisticGen, (datObsList[i], RNu, RWIMP, NuUncs, exposure))\
                                             for i in range(n)]
        pool.close()
        pool.join()
        H1Stat= [res.get() for res in multiple_results]
        #H1Stat = [self._statisticGen(datObsList[i], RNu, sigma0, Uncs, exposure) for i in range(n)]
        time_end = time.perf_counter()
        print("Time costed: {0} s.".format(time_end-time_start))
        return H1Stat

In [2]:
EthList = [1e-4,3e-3,3e-3,4.,4.,4.]
sigma0List = np.array([-43.8, -44.19, -44.25, -48.34, -48.7, -47.67])
exposureList = np.array([0.018, 0.018, 0.018, 9000.,9000.,9000.])
selList = []
phiList=[6.4,11.3,17.4,5.93,10.9,18.3]
massList=[0.2,2.,5.5,10.,100.,1000.]

In [3]:
MCList = [MCCheck(massList[i], EthList[i], E_max, ne) for i in range(6)]

In [4]:
H1DatList = [MCList[i].H1StatGen(sigma0List[i], exposureList[i], 10000, selList)\
              for i in range(6)]

Time costed: 9144.738319643977 s.
Time costed: 6575.338029829989 s.
Time costed: 6584.72966699602 s.
Time costed: 872.1881686959532 s.
Time costed: 967.864014592953 s.
Time costed: 967.7402998589678 s.


In [5]:
mypath = os.path.join(os.path.abspath(os.path.pardir),'data', 'MC')
myData = H1DatList
myFile = open(os.path.join(mypath,'MCNu.pickle'), 'wb')
pickle.dump(myData, myFile)
myFile.close()